In [ ]:
import pandas as pd

#For Kaggle
#date=''

#For WL data
date='20241118'
source='WL'

saveImg=False

print('done')

dfTrxSaved = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrxSaved.head(5)

In [ ]:
sorted_df = dfTrxSaved.sort_values(by=['card_pan_id','trx_date_time'])
sorted_df=sorted_df.drop(columns=['mcd_fraud_score','vaa_score',
                                 'trx_amount_log10','ecom',	'card_brand_MCD',	'card_brand_VIS','country_group_BELGIUM','country_group_EUROPE',
                                 'clusterCardHolder_cluster0',	'clusterCardHolder_cluster1',	'clusterCardHolder_cluster2',	'clusterCardHolder_cluster3',	
                                 'clusterCardHolder_cluster4',	'clusterMerchant_M0',	'clusterMerchant_M1','clusterMerchant_M2','clusterMerchant_M3',
                                 'clusterMerchant_UNKNOWN',
                                  'country_group_WORLD','mcc_group_CLOTHING_STORES',	
                                  'mcc_group_GOUVERNEMENT_SERVICES','mcc_group_OTHER','mcc_group_PROFESSIONAL_SERVICES',	
                                  'mcc_group_RETAIL_OUTLET_SERVICES','mcc_group_UTILITY_SERVICES',	
                                  'trx_reversal_FULL_REVERSAL',	'trx_reversal_NO_RESERSAL',	
                                  'trx_reversal_PARTIAL_REVERSAL','clusterCardHolder_UNKNOWN'
                                 ])
#print(sorted_df.head(10))
#print(sorted_df.shift(-1).head(10))


sorted_df['card_pan_id1'] = sorted_df['card_pan_id'].shift(-1)
sorted_df['term_mcc1'] = sorted_df['term_mcc'].shift(-1)
sorted_df['term_country1'] = sorted_df['term_country'].shift(-1)
sorted_df.head(10)

sorted_df['card_pan_id2'] = sorted_df['card_pan_id'].shift(-2)
sorted_df['term_mcc2'] = sorted_df['term_mcc'].shift(-2)
sorted_df['term_country2'] = sorted_df['term_country'].shift(-2)

sorted_df['card_pan_id3'] = sorted_df['card_pan_id'].shift(-3)
sorted_df['term_mcc3'] = sorted_df['term_mcc'].shift(-3)
sorted_df['term_country3'] = sorted_df['term_country'].shift(-3)

sorted_df['card_pan_id4'] = sorted_df['card_pan_id'].shift(-4)
sorted_df['term_mcc4'] = sorted_df['term_mcc'].shift(-4)
sorted_df['term_country4'] = sorted_df['term_country'].shift(-4)

sorted_df['card_pan_id5'] = sorted_df['card_pan_id'].shift(-5)
sorted_df['term_mcc5'] = sorted_df['term_mcc'].shift(-5)
sorted_df['term_country5'] = sorted_df['term_country'].shift(-5)

sorted_df['card_pan_id6'] = sorted_df['card_pan_id'].shift(-6)
sorted_df['term_mcc6'] = sorted_df['term_mcc'].shift(-6)
sorted_df['term_country6'] = sorted_df['term_country'].shift(-6)

sorted_df['card_pan_id7'] = sorted_df['card_pan_id'].shift(-7)
sorted_df['term_mcc7'] = sorted_df['term_mcc'].shift(-7)
sorted_df['term_country7'] = sorted_df['term_country'].shift(-7)


In [ ]:
sorted_df1=sorted_df[(sorted_df['card_pan_id']==sorted_df['card_pan_id1'])]
sorted_df2=sorted_df[(sorted_df['card_pan_id']==sorted_df['card_pan_id2'])]
sorted_df3=sorted_df[(sorted_df['card_pan_id']==sorted_df['card_pan_id3'])]
sorted_df4=sorted_df[(sorted_df['card_pan_id']==sorted_df['card_pan_id4'])]
sorted_df5=sorted_df[(sorted_df['card_pan_id']==sorted_df['card_pan_id5'])]
sorted_df6=sorted_df[(sorted_df['card_pan_id']==sorted_df['card_pan_id6'])]
sorted_df7=sorted_df[(sorted_df['card_pan_id']==sorted_df['card_pan_id7'])]

print(sorted_df.shape)
print(sorted_df1.shape)
print(sorted_df2.shape)
print(sorted_df3.shape)
print(sorted_df4.shape)
print(sorted_df5.shape)
print(sorted_df6.shape)
print(sorted_df7.shape)
print(sorted_df.Class.value_counts(normalize=True))
print(sorted_df1.Class.value_counts(normalize=True))
print(sorted_df2.Class.value_counts(normalize=True))
print(sorted_df3.Class.value_counts(normalize=True))
print(sorted_df4.Class.value_counts(normalize=True))
print(sorted_df5.Class.value_counts(normalize=True))
print(sorted_df6.Class.value_counts(normalize=True))
print(sorted_df7.Class.value_counts(normalize=True))

#sorted_df4.head(50)

In [ ]:
import numpy as np

dfTrxSaved['previous_trx']=0
for i in range(1,2,1):
    sorted_df['card_pan_id1'] = sorted_df['card_pan_id'].shift(-i)
    sorted_dfTemp=sorted_df[(sorted_df['card_pan_id']==sorted_df['card_pan_id1'])]
    dfTrxSaved['previous_trx']=np.where(sorted_df.index.isin(sorted_dfTemp.index),i,dfTrxSaved['previous_trx'])
    



#sorted_df['card_pan_id2'] = sorted_df['card_pan_id'].shift(-2)
#sorted_dfTemp=sorted_df[(sorted_df['card_pan_id']==sorted_df['card_pan_id2'])]
#dfTrxSaved['previous_trx']=np.where(sorted_df.index.isin(sorted_dfTemp.index),2,dfTrxSaved['previous_trx'] )

dfTrxSaved['previous_trx'].value_counts()

In [ ]:
dfTrxSaved.head()

In [ ]:
for i in range(0,3,1):
    print(i)
    print(dfTrxSaved[(dfTrxSaved['previous_trx']==i)].shape[0])
    print(dfTrxSaved['Class'][(dfTrxSaved['previous_trx']==i)].value_counts(normalize=True))

In [ ]:
print(dfTrxSaved['Class'].value_counts(normalize=True))
print(dfTrxSaved['Class'][(dfTrxSaved['clusterCardHolder_cluster0']==1) & (dfTrxSaved['previous_trx']==1)].value_counts(normalize=True))
print(dfTrxSaved['Class'][(dfTrxSaved['clusterCardHolder_cluster1']==1) & (dfTrxSaved['previous_trx']==1)].value_counts(normalize=True))
print(dfTrxSaved['Class'][(dfTrxSaved['clusterCardHolder_cluster2']==1) & (dfTrxSaved['previous_trx']==1)].value_counts(normalize=True))
print(dfTrxSaved['Class'][(dfTrxSaved['clusterCardHolder_cluster3']==1) & (dfTrxSaved['previous_trx']==1)].value_counts(normalize=True))
print(dfTrxSaved['Class'][(dfTrxSaved['clusterCardHolder_cluster4']==1) & (dfTrxSaved['previous_trx']==1)].value_counts(normalize=True))